# Import Libraries

In [93]:
#import all required libraries
from decimal import Decimal
from io import BytesIO
import json
import logging
import os
from pprint import pprint
import requests
from zipfile import ZipFile
from boto3.dynamodb.conditions import Key
from boto3 import Session
from botocore.exceptions import ClientError
from configparser import ConfigParser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
from datetime import date
import pandas as pd
import re

from src.beer_functions import Beers

In [94]:
session = Session(profile_name='ds1')
dyn_resource = session.resource("dynamodb")
table = dyn_resource.Table('md-brewery')

In [51]:
table = dyn_resource.create_table(TableName = 'md-brewery',
                              KeySchema=[
                    {"AttributeName": "scraping_date", "KeyType": "HASH"},  # Partition key
                    {"AttributeName": "beer_name", "KeyType": "RANGE"},  # Sort key
                ],
                AttributeDefinitions=[
                    {"AttributeName": "scraping_date", "AttributeType": "S"},
                    {"AttributeName": "beer_name", "AttributeType": "S"},
                ],
                ProvisionedThroughput={
                    "ReadCapacityUnits": 5,
                    "WriteCapacityUnits": 5,
                },)
table.wait_until_exists()

In [95]:
with open("beer_xpaths.json") as file:
    beer_json = json.load(file)

In [96]:
for key in beer_json.keys():
    print(key)

cushwa
blackflag


In [97]:
beer_json['cushwa']

{'name': 'Cushwa Brewing',
 'url': 'https://cushwabrewing.com/columbia',
 'authentication': {'name': 'overAge'},
 'beer_attrs': {'beers': {'xpath': "//html//div[@class='item']//h4//span[@id]"},
  'style': {'xpath': "//html//div[@class='item']//h4//span[@class='item-category']"},
  'description': {'xpath': "//html//div[@class='item']//p[@class='link-font-color show-less item-title-color']"},
  'abv': {'classname': 'item-abv'},
  'brewery': {'classname': 'brewery'}}}

In [98]:
if beer_json['cushwa']['authentication'] is False:
    print('yes')
else:
    print('no')

no


In [92]:
beer_json.keys()

dict_keys(['cushwa', 'blackflag'])

In [89]:
len(beer_json.items())

2

In [112]:
def scrape_info(attribute_dict: dict) ->list:
    """returns the text of a webpage using the argument scraping_method for
    path scraping_path"""
    assert len(attribute_dict.items())==1
    scraping_method = list(attribute_dict.keys())[0]
    scraping_path = list(attribute_dict.values())[0]
    if scraping_method.lower().startswith('class'):
        try:
            text = [col.text for col in driver.find_elements(By.CLASS_NAME,
                                                             scraping_path)]
        except ValueError:
            pass
    elif scraping_method.lower().startswith("xpath"):
        try:
            text = [col.text for col in driver.find_elements(By.XPATH,
                                                             scraping_path)]
        except ValueError:
            pass
    return text
    
    # [col.text for col in driver.find_elements(By.CLASS_NAME,'item-abv')]
    # pass

In [100]:
def clean_abv(raw_abv: str)-> str:
    abv_final = re.match(pattern = "[0-9]+[.]*[0-9]*",
                               string = raw_abv).group(0)
    return abv_final

In [123]:
for table in dyn_resource.tables.all():
                print(table.name)

md-brewery


In [101]:
# Chromedriver located in /usr/local/bin

driver = webdriver.Chrome()

# Create Class for brewery scraping

In [13]:
class Brewery:
    def __init__(self,name,url):
        self.name = name
        self.url = url
    def __str__(self):
        return self.name
    def __repr__(self):
        return f"Brewery({self.name}, {self.url})"
cushwa = Brewery('Cushwa Brewing', "https://cushwabrewing.com/columbia")
driver.get(cushwa.url)
# button = driver.find_element(By.NAME, 'overAge')
# button.click()

In [102]:
def authenticate_age(scraping_path: str):
    try:
        button = driver.find_element(By.NAME, scraping_path)
        return button.click()
    except:
        button = button = driver.find_element(By.XPATH, scraping_path)
        return button.click()


In [103]:
beer_json["cushwa"]

{'name': 'Cushwa Brewing',
 'url': 'https://cushwabrewing.com/columbia',
 'authentication': {'name': 'overAge'},
 'beer_attrs': {'beers': {'xpath': "//html//div[@class='item']//h4//span[@id]"},
  'style': {'xpath': "//html//div[@class='item']//h4//span[@class='item-category']"},
  'description': {'xpath': "//html//div[@class='item']//p[@class='link-font-color show-less item-title-color']"},
  'abv': {'classname': 'item-abv'},
  'brewery': {'classname': 'brewery'}}}

In [105]:
driver.get(beer_json["cushwa"]["url"])

In [106]:
authenticate_age("overAge")

In [111]:
for key in beer_json['cushwa']['beer_attrs'].keys():
    print(beer_json['cushwa']['beer_attrs'][key])

{'xpath': "//html//div[@class='item']//h4//span[@id]"}
{'xpath': "//html//div[@class='item']//h4//span[@class='item-category']"}
{'xpath': "//html//div[@class='item']//p[@class='link-font-color show-less item-title-color']"}
{'classname': 'item-abv'}
{'classname': 'brewery'}


In [131]:
beer_dict = dict()
for key in beer_json['cushwa']['beer_attrs'].keys():
    if key.lower() != 'abv':
        beer_dict[key] = scrape_info(beer_json['cushwa']["beer_attrs"][key])
    else:
        beer_dict[key] = [clean_abv(abv) \
                          for abv in scrape_info(beer_json['cushwa'] \
                                                 ["beer_attrs"][key])]

beer_dict["scraping_date"] = date.today().strftime("%m/%d/%Y")

In [132]:
pd.DataFrame(beer_dict)

,beers,style,description,abv,brewery,scraping_date
0,X Marks the Spot,Lager - Mexican,This Mexican Style Amber pours a beautiful lig...,5.4,Cushwa Brewing Co.,05/09/2024
1,Mai the 4th Be With You,Maibock,"Maibock, a pale, malt driven lager. With notes...",6.7,Cushwa Brewing Co.,05/09/2024
2,Starting Over,Lager - Dortmunder,We’re always excited to go back into the beer ...,5,Cushwa Brewing Co.,05/09/2024
3,That'll Play,Lager - American Light,"Sky’s out limes out. That’ll Play is a clean, ...",4.2,Cushwa Brewing Co.,05/09/2024
4,That’ll Play Masters Edition,Lager - American Light,We took our That’ll Play lager and gave it the...,4.2,Cushwa Brewing Co.,05/09/2024
5,Rad Beer,Lager - American Light,When you have a pizza as good as the World Cha...,4.8,Cushwa Brewing Co.,05/09/2024
6,Kolschwa,Kölsch,Brewed traditionally to the German style,5,Cushwa Brewing Co.,05/09/2024
7,Pura Pils,Pilsner - Other,"An all-day crusher, packed full of flavor that...",5.5,Cushwa Brewing Co.,05/09/2024
8,Going Bridge,Red Ale - Irish,"Predominantly Irish malt, and is fermented wit...",4.8,Cushwa Brewing Co.,05/09/2024
9,Wit,Witbier,This is a very traditional Belgian Witbier (no...,5.7,Cushwa Brewing Co.,05/09/2024


In [20]:
driver.quit()

## Write each item to dynamodb table

In [133]:
with table.batch_writer() as writer:
    for num in range(len(beer_dict['beers'])):
        writer.put_item(Item={
        "scraping_date": beer_dict["scraping_date"],
        "brewery": beer_dict["brewery"][num],
        "beer_name": beer_dict["beers"][num],
        "style": beer_dict["style"][num],
        "abv": Decimal(str(beer_dict["abv"][num])),
        "description": beer_dict["description"][num]
        }
        )


In [64]:
for num in range(len(beer_dict['name'])):
    table.put_item(
                Item={
                    "scraping_date": beer_dict["scraping_date"],
                    "brewery": beer_dict["maker"][num],
                    "beer_name": beer_dict["name"][num],
                    "style": beer_dict["style"][num],
                    "abv": Decimal(str(beer_dict["abv"][num])),
                    "description": beer_dict["description"][num]
                    }
            )

# Elder Pine

In [44]:
driver.get("https://elderpine.com/tasting-room")

In [45]:
[col.text for col in driver.find_elements(By.XPATH,"//html//div[@class='row sqs-row']//p[@style='text-align:center;white-space:pre-wrap;']")]

['EP Pils',
 'Classic Pilsner',
 '5% ABV',
 'Dunkel Wood',
 'Oak-aged Munich-style Dark Lager ',
 '5.2% ABV',
 'Dense Brume',
 'Hazy IPA',
 '7% ABV',
 'The Midnight Sun',
 'Barrel-aged Dark Sour Ale',
 '9% ABV',
 'The Slender Sun Receding',
 'Barrel-aged Sour Citrus Ale',
 '7% ABV',
 'Lemon Lime Reflections',
 'Fruited Belgian-style Witbier',
 '5% ABV',
 'Asleep on the West Coast',
 'West Coast-style Double IPA',
 '8.7% ABV',
 'Cognitive Relections',
 'Classic Belgian-style Witbier',
 '5% ABV',
 'COMING SOON',
 'COMING SOON',
 'COMING SOON',
 'Pliable Foe',
 'Coffee Dark Mild',
 '3.8% ABV',
 'Sent Forth From The Eye Of The Storm',
 '',
 'Barrel-aged Boysenberry Sour Ale',
 '8% ABV',
 'Barrel-Aged Sparrow’s Fall',
 'Vienna-style Lager',
 '5% ABV',
 'Barrel-aged Thorunn The Shield Maiden',
 'Inside Bar Only',
 'Norwegian-inspired Helles Bock Lager',
 '8% ABV',
 'Abject Terror',
 'Inside Bar Only',
 'Oak-aged Barleywine-style Lager ',
 '12% ABV',
 'The Otter In The Rye',
 'Barrel-aged Rye

In [46]:
elderpine = Brewery("Elder Pine Brewing", url = "https://elderpine.com/tasting-room")

In [47]:
elderpine.beers = [col.text for col in driver.find_elements(By.XPATH,"//div[@class='sqs-html-content']/p[contains(@style,'text')]/strong")
                   if re.search("[a-zA-Z]+", col.text) and not col.text.lower().startswith('coming soon')]

In [48]:
elderpine.beers

['EP Pils',
 'Dunkel Wood',
 'Dense Brume',
 'The Midnight Sun',
 'The Slender Sun Receding',
 'Lemon Lime Reflections',
 'Asleep on the West Coast',
 'Cognitive Relections',
 'Pliable Foe',
 'Sent Forth From The Eye Of The Storm',
 'Barrel-Aged Sparrow’s Fall',
 'Barrel-aged Thorunn The Shield Maiden',
 'Abject Terror',
 'The Otter In The Rye',
 'Lord Maris, The Otter ',
 'NITRO Bitter Geezer ',
 'NITRO Barrel-aged Nut Brown']

In [38]:
beer2 = pd.DataFrame(beer_df)

In [49]:
driver.quit()

In [89]:
len(elderpine.beers)

19

# Removing the first beer in the list

## The page is using different tags for the first beer abv and style

In [90]:
beer_df = {'name':elderpine.beers[1:],
                        "style":elderpine.style,
              'abv': elderpine.abv,
              }

In [91]:
pd.DataFrame(beer_df)

,name,style,abv
0,Rescue,German-style Pilsner,4.9
1,Empyreumatic,Smoked Helles Lager,5.5
2,Barrel-aged Imperator,Imperial Rauchbier,11.0
3,Continuous Refinement #2,West Coast IPA,6.2
4,Chin Chin!,Irish-style Extra Stout,7.0
5,Asleep on the West Coast,West Coast-style Double IPA,8.7
6,Dunkel Wood,Oak-aged Munich-style Dark Lager,5.2
7,In Sunshine & In Shadow,Hazy Double IPA,8.0
8,Barrel-aged Gjaerkauk,Norwegian-style Smoked Amber Ale,7.2
9,"Barrel-aged Praamzius, Ruler Of Time",Baltic Porter,10.5


In [85]:
print(len(elderpine.abv)) 
print(len(elderpine.style))
print(len(elderpine.beers))

18
18
25


In [86]:
elderpine.beers

['EP Pils',
 'Rescue',
 'Empyreumatic',
 'Barrel-aged Imperator',
 'Continuous Refinement #2',
 'Chin Chin!',
 'Asleep on the West Coast',
 'Dunkel Wood',
 'In Sunshine & In Shadow',
 'Barrel-aged Gjaerkauk',
 'Barrel-aged Praamzius, Ruler Of Time',
 'Jupiter’s Cake',
 'Barrel-aged Thorunn The Shield Maiden',
 'Abject Terror',
 'Southern Hemisphere Haze',
 'Proper Porter',
 'Lord Maris, The Otter ',
 ' ',
 'NITRO Barrel-aged Nut Brown',
 ' ',
 'NITRO Bitter Geezer ',
 'COMING SOON!',
 'Coming Soon!',
 'Coming Soon!',
 'Coming Soon!']

In [23]:
styles = [col.text for col in driver.find_elements(By.XPATH,r"//html//div[@class='row sqs-row']//p[@style='text-align:center;white-space:pre-wrap;']//em")]

In [26]:
# Only getting the 2nd beer onwards
grouped_attr = [col.text for col in driver.find_elements(By.XPATH,"//div[@class='sqs-html-content']//p[@style='text-align:center;white-space:pre-wrap;']//em")]

In [75]:
if not re.search("ABV",grouped_attr[0]):
    print("no match")
else:
    print('found a match')

no match


In [44]:
elderpine.abv = [clean_abv(row) for row in grouped_attr if row.__contains__(" ABV")]
elderpine.style = [row.strip() for row in grouped_attr if re.findall("^[a-zA-Z]+",row)
                   and row.lower() != "inside bar only"]


In [69]:
assert len(elderpine.abv) == len(elderpine.style)

In [76]:
assert len(elderpine.name) == len(elderpine.style)

# Other Half Brewing

In [92]:
otherhalf = Brewery('Other Half Brewing', "https://otherhalfbrewing.com/our-beers")
driver.get(otherhalf.url)
button = driver.find_element(By.XPATH, '//button[@id="age-gate-pass"]')
button.click()

In [39]:
driver.maximize_window()

In [94]:
driver.quit()

# Falling Branch Brewing